<a href="https://colab.research.google.com/github/Hyun-ho-Lee/2021-DACON-Credit_Card/blob/main/Credit_card_2021_05_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import random
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.impute import SimpleImputer

# model
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

sns.set_style('darkgrid')

import os 
from google.colab import drive
drive.mount('/content/drvie')

Drive already mounted at /content/drvie; to attempt to forcibly remount, call drive.mount("/content/drvie", force_remount=True).


In [ ]:
train=pd.read_csv("/content/drvie/MyDrive/traindata.csv")
test=pd.read_csv("/content/drvie/MyDrive/test.csv")

In [ ]:
# train
train_index = train['index']
train.drop('index', inplace = True, axis = 1)

# test
#test = pd.read_csv('test.csv')
test_index = test['index']
test.drop('index', inplace = True, axis = 1)

In [ ]:
train.fillna(np.NaN, inplace = True)
test.fillna(np.NaN, inplace = True)

In [ ]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [ ]:
# Label Encoding
le = LabelEncoder()
le_count = 0

for col in train:
    if train[col].dtype == 'object':
        # unique <= 2
        if len(list(train[col].unique())) <= 2:
            le.fit(train[col])
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])
            
            le_count += 1
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [ ]:
# One-Hot Encoding
train = pd.get_dummies(train)
test = pd.get_dummies(test)

print('Train Features shape: ', train.shape)
print('Test Features shape: ', test.shape)

Train Features shape:  (26457, 53)
Test Features shape:  (10000, 52)


In [ ]:
def days_to_age(x):
    return (x*-1)/365


train['age']=days_to_age(train['DAYS_BIRTH'])


In [ ]:
test['age']=days_to_age(test['DAYS_BIRTH'])

In [ ]:
train.drop(["DAYS_BIRTH","DAYS_EMPLOYED","age"],axis=1)

In [ ]:
bins=[0,20,30,40,50,60,70]
np.digitize(train['age'], bins)
train['age_cat']=np.digitize(train['age'], bins)

In [ ]:
np.digitize(test['age'], bins)
test['age_cat']=np.digitize(test['age'], bins)

In [ ]:
np.digitize(test['age'], bins)
test['age_cat']=np.digitize(test['age'], bins)

In [ ]:
def days_to_work(x):
    return (x*-1)/365

train['work_year']=days_to_work(train["DAYS_EMPLOYED"])

In [ ]:
def days_to_work(x):
    return (x*-1)/365

test['work_year']=days_to_work(test["DAYS_EMPLOYED"])

In [ ]:
train['work_year']=train['work_year'].round(1) #애매한 반올림 제거 
train['work_year']
train['work_year'].describe()
#0이하를 무직자로 생각
train.loc[(train.work_year<0),'work_year']=0

In [ ]:
test['work_year']=test['work_year'].round(1) #애매한 반올림 제거 
test['work_year']
test['work_year'].describe()
#0이하를 무직자로 생각
test.loc[(test.work_year<0),'work_year']=0

In [ ]:
train['work_year'].describe()

count    26457.000000
mean         6.023102
std          6.493439
min          0.000000
25%          1.100000
50%          4.200000
75%          8.600000
max         43.000000
Name: work_year, dtype: float64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 56 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender                                  26457 non-null  int64  
 1   car                                     26457 non-null  int64  
 2   reality                                 26457 non-null  int64  
 3   child_num                               26457 non-null  int64  
 4   income_total                            26457 non-null  float64
 5   DAYS_BIRTH                              26457 non-null  int64  
 6   DAYS_EMPLOYED                           26457 non-null  int64  
 7   FLAG_MOBIL                              26457 non-null  int64  
 8   work_phone                              26457 non-null  int64  
 9   phone                                   26457 non-null  int64  
 10  email                                   26457 non-null  in

In [ ]:
train=train.drop(["age","DAYS_BIRTH","DAYS_EMPLOYED"],axis=1)

In [ ]:
test=test.drop(["age","DAYS_BIRTH","DAYS_EMPLOYED"],axis=1)

In [ ]:
train

,gender,car,reality,child_num,income_total,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,age_cat,work_year
0,0,0,0,0,202500.0,1,0,0,0,2.0,-6.0,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,12.9
1,0,0,1,1,247500.0,1,0,0,1,3.0,-5.0,1.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4.2
2,1,1,1,0,450000.0,1,0,1,0,2.0,-22.0,2.0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,12.1
3,0,0,1,0,202500.0,1,0,1,0,2.0,-37.0,0.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4,5.7
4,0,1,1,0,157500.0,1,0,0,0,2.0,-26.0,2.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,4,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,1,0,0,0,4.0,-2.0,1.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5.4
26453,0,0,1,1,180000.0,1,0,0,0,2.0,-47.0,2.0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6.8
26454,0,1,0,0,292500.0,1,0,0,0,2.0,-25.0,2.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5.5
26455,1,0,1,0,171000.0,1,0,0,0,1.0,-59.0,2.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.3


In [ ]:
test

,gender,car,reality,child_num,income_total,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,age_cat,work_year
0,1,1,0,0,112500.0,1,0,1,0,2.0,-60.0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0.0
1,0,0,1,0,135000.0,1,0,1,0,2.0,-36.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,23.8
2,0,0,1,0,69372.0,1,1,1,0,2.0,-40.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4,0.6
3,1,1,0,0,112500.0,1,1,0,0,2.0,-41.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,6.9
4,0,1,1,0,225000.0,1,1,0,0,2.0,-8.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,4,25.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,0,202500.0,1,1,1,0,2.0,-19.0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,14.9
9996,1,1,1,0,202500.0,1,1,0,0,2.0,-34.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,3.6
9997,0,0,1,0,292500.0,1,0,0,0,2.0,-55.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,38.4
9998,0,1,0,0,180000.0,1,0,1,0,2.0,-33.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3.0


In [ ]:
pd.get_dummies(data = train, columns = ['age_cat'], prefix = 'age_cat')

,gender,car,reality,child_num,income_total,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,work_year,age_cat_2,age_cat_3,age_cat_4,age_cat_5,age_cat_6
0,0,0,0,0,202500.0,1,0,0,0,2.0,-6.0,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12.9,0,1,0,0,0
1,0,0,1,1,247500.0,1,0,0,1,3.0,-5.0,1.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4.2,0,1,0,0,0
2,1,1,1,0,450000.0,1,0,1,0,2.0,-22.0,2.0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,12.1,0,0,0,1,0
3,0,0,1,0,202500.0,1,0,1,0,2.0,-37.0,0.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5.7,0,0,1,0,0
4,0,1,1,0,157500.0,1,0,0,0,2.0,-26.0,2.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5.8,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,1,0,0,0,4.0,-2.0,1.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.4,0,1,0,0,0
26453,0,0,1,1,180000.0,1,0,0,0,2.0,-47.0,2.0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.8,0,0,1,0,0
26454,0,1,0,0,292500.0,1,0,0,0,2.0,-25.0,2.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.5,1,0,0,0,0
26455,1,0,1,0,171000.0,1,0,0,0,1.0,-59.0,2.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.3,1,0,0,0,0


In [ ]:
pd.get_dummies(data = test, columns = ['age_cat'], prefix = 'age_cat')

,gender,car,reality,child_num,income_total,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,work_year,age_cat_2,age_cat_3,age_cat_4,age_cat_5,age_cat_6
0,1,1,0,0,112500.0,1,0,1,0,2.0,-60.0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,1
1,0,0,1,0,135000.0,1,0,1,0,2.0,-36.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23.8,0,0,0,1,0
2,0,0,1,0,69372.0,1,1,1,0,2.0,-40.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.6,0,0,1,0,0
3,1,1,0,0,112500.0,1,1,0,0,2.0,-41.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6.9,0,0,0,1,0
4,0,1,1,0,225000.0,1,1,0,0,2.0,-8.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,25.7,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,0,202500.0,1,1,1,0,2.0,-19.0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.9,0,0,0,1,0
9996,1,1,1,0,202500.0,1,1,0,0,2.0,-34.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.6,1,0,0,0,0
9997,0,0,1,0,292500.0,1,0,0,0,2.0,-55.0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,38.4,0,0,0,1,0
9998,0,1,0,0,180000.0,1,0,1,0,2.0,-33.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0,0,1,0,0


In [ ]:
train['begins_month']=(train['begin_month'] / -12)

In [ ]:
test['begins_month']=(test['begin_month'] / -12)

In [ ]:
train['begins_month'].describe()

count    26457.000000
mean         2.176941
std          1.379962
min         -0.000000
25%          1.000000
50%          2.000000
75%          3.250000
max          5.000000
Name: begins_month, dtype: float64

In [ ]:
train=train.drop(["begin_month"],axis=1)

In [ ]:
test=test.drop(["begin_month"],axis=1)

In [ ]:
train

,gender,car,reality,child_num,income_total,FLAG_MOBIL,work_phone,phone,email,family_size,credit,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,age_cat,work_year,begins_month
0,0,0,0,0,202500.0,1,0,0,0,2.0,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,12.9,0.500000
1,0,0,1,1,247500.0,1,0,0,1,3.0,1.0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4.2,0.416667
2,1,1,1,0,450000.0,1,0,1,0,2.0,2.0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,12.1,1.833333
3,0,0,1,0,202500.0,1,0,1,0,2.0,0.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4,5.7,3.083333
4,0,1,1,0,157500.0,1,0,0,0,2.0,2.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,4,5.8,2.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,1,0,0,0,4.0,1.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5.4,0.166667
26453,0,0,1,1,180000.0,1,0,0,0,2.0,2.0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6.8,3.916667
26454,0,1,0,0,292500.0,1,0,0,0,2.0,2.0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5.5,2.083333
26455,1,0,1,0,171000.0,1,0,0,0,1.0,2.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0.3,4.916667


Modeling 

In [ ]:
pip install pycaret

In [ ]:
import random
from pycaret.classification import * 
from sklearn.metrics import log_loss

In [ ]:
clf = setup(train, target = 'credit', train_size = 0.85)

,Description,Value
0,session_id,5885
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 53)"
5,Missing Values,False
6,Numeric Features,43
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7ffbc5756560>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
best5 = compare_models(fold = 10, sort = 'logloss', n_select = 5, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6963,0.7067,0.4243,0.6917,0.6184,0.2330,0.3175,0.7665,0.786
gbc,Gradient Boosting Classifier,0.6909,0.6472,0.4093,0.6713,0.6045,0.2075,0.3008,0.7996,9.776
lda,Linear Discriminant Analysis,0.6422,0.6067,0.3378,0.5387,0.5088,0.0131,0.0527,0.8649,0.306
nb,Naive Bayes,0.6407,0.5686,0.3333,0.4105,0.5004,0.0000,0.0000,0.8855,0.068
lr,Logistic Regression,0.6407,0.5103,0.3333,0.4105,0.5004,0.0000,0.0000,0.9204,0.505
rf,Random Forest Classifier,0.6921,0.7514,0.5372,0.6715,0.6776,0.3631,0.3676,1.0509,3.657
ada,Ada Boost Classifier,0.6904,0.6206,0.4053,0.6431,0.6006,0.1995,0.3012,1.0811,0.956
et,Extra Trees Classifier,0.6665,0.7101,0.5223,0.6489,0.6551,0.3199,0.3225,2.9967,4.141
knn,K Neighbors Classifier,0.6290,0.6748,0.4702,0.6049,0.6131,0.2322,0.2355,3.7730,0.811
dt,Decision Tree Classifier,0.6046,0.6388,0.4932,0.6159,0.6097,0.2562,0.2566,13.1678,0.271


In [ ]:
blended = blend_models(estimator_list = best5, fold = 10, optimize = 'logloss')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6856,0.6606,0.3971,0.6162,0.5915,0.1805,0.2862,0.8267
1,0.6865,0.6554,0.3984,0.6139,0.5930,0.1843,0.2887,0.8246
2,0.6936,0.6716,0.4088,0.6227,0.6050,0.2094,0.3127,0.8199
3,0.6923,0.6497,0.4062,0.6346,0.6018,0.1993,0.3123,0.8259
4,0.6945,0.6736,0.4097,0.6316,0.6059,0.2092,0.3180,0.8174
5,0.6919,0.6760,0.4067,0.6179,0.6026,0.2048,0.3059,0.8155
6,0.6883,0.6773,0.4010,0.6161,0.5959,0.1905,0.2951,0.8201
7,0.6910,0.6512,0.4060,0.6205,0.6013,0.2008,0.3036,0.8274
8,0.6913,0.6575,0.4057,0.6248,0.6011,0.1999,0.3064,0.8222
9,0.6824,0.6598,0.3931,0.6153,0.5862,0.1684,0.2755,0.8285


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6916,0,0.3976,0.6284,0.5997,0.1874,0.294,0


In [ ]:
custom_model = compare_models(fold = 10, sort = 'logloss', n_select = 6, include=['lightgbm','gbc','rf','ada', 'lda', 'nb'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6963,0.7067,0.4243,0.6917,0.6184,0.2330,0.3175,0.7665,0.737
gbc,Gradient Boosting Classifier,0.6909,0.6472,0.4093,0.6713,0.6045,0.2075,0.3008,0.7996,9.785
lda,Linear Discriminant Analysis,0.6422,0.6067,0.3378,0.5387,0.5088,0.0131,0.0527,0.8649,0.307
nb,Naive Bayes,0.6407,0.5686,0.3333,0.4105,0.5004,0.0000,0.0000,0.8855,0.073
rf,Random Forest Classifier,0.6921,0.7514,0.5372,0.6715,0.6776,0.3631,0.3676,1.0509,3.549
ada,Ada Boost Classifier,0.6904,0.6206,0.4053,0.6431,0.6006,0.1995,0.3012,1.0811,0.951


In [ ]:
blended_custom = blend_models(estimator_list = custom_model, fold = 10, optimize = 'logloss')
pred_holdout_custom = predict_model(blended_custom)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6879,0.7437,0.4019,0.6135,0.5971,0.1929,0.2916,0.7926
1,0.6896,0.7413,0.4063,0.6949,0.6013,0.2005,0.2972,0.7890
2,0.6999,0.7562,0.4194,0.6880,0.6164,0.2339,0.3314,0.7837
3,0.6968,0.7449,0.4154,0.7578,0.6115,0.2187,0.3241,0.7875
4,0.7008,0.7517,0.4194,0.6953,0.6166,0.2330,0.3366,0.7827
5,0.6994,0.7480,0.4193,0.6866,0.6162,0.2333,0.3298,0.7843
6,0.6941,0.7544,0.4121,0.6960,0.6076,0.2133,0.3138,0.7843
7,0.6950,0.7305,0.4144,0.6834,0.6102,0.2188,0.3153,0.7927
8,0.6944,0.7448,0.4111,0.6863,0.6070,0.2130,0.3157,0.7893
9,0.6855,0.7456,0.3995,0.7415,0.5928,0.1806,0.2863,0.7935


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6974,0.749,0.4075,0.7198,0.6106,0.209,0.3135,0.7782


In [ ]:
#자동으로 상위 5개 선택한 모델
final_model = finalize_model(blended)

#Accurary, AUC, Logloss 셋다 상위 4개인 모델 
final_model_custom = finalize_model(blended_custom)

In [ ]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model_custom])
prections = prep_pipe.predict_proba(test)
prections


array([[0.12980444, 0.20160275, 0.66859281],
       [0.21788696, 0.20936363, 0.5727494 ],
       [0.1214883 , 0.18483407, 0.69367763],
       ...,
       [0.10857406, 0.1338783 , 0.75754764],
       [0.21499057, 0.2254327 , 0.55957673],
       [0.1456248 , 0.27014387, 0.58423133]])

In [ ]:
i, j, k = [], [], []
for row in prections:
  i.append(row[0])
  j.append(row[1])
  k.append(row[2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [ ]:
submit=pd.read_csv("/content/drvie/MyDrive/sample_submission.csv")

TypeError: ignored

In [ ]:
submit['0'] = i
submit['1'] = j
submit['2'] = k
submit.head(10)

,index,0,1,2
0,26457,0.129804,0.201603,0.668593
1,26458,0.217887,0.209364,0.572749
2,26459,0.121488,0.184834,0.693678
3,26460,0.129610,0.181314,0.689076
4,26461,0.151672,0.237735,0.610593
5,26462,0.137464,0.246219,0.616317
6,26463,0.281477,0.441077,0.277446
7,26464,0.142315,0.222836,0.634849
8,26465,0.138938,0.198822,0.662240
9,26466,0.129747,0.241161,0.629092


In [ ]:
submit.to_csv("submission.csv",index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')